## PromptTemplate


In [54]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


LLM 객체를 정의합니다.


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

### 방법 1. from_template() 메소드를 사용하여 PromptTemplate 객체 생성

- 치환될 변수를 `{ 변수 }` 로 묶어서 템플릿을 정의합니다.


In [56]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

`country` 변수에 값을 넣어서 문장을 생성할 수 있습니다.


In [57]:
# prompt 생성. format 메소드를 이용하여 변수에 값을 넣어줌
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [58]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [59]:
# country 변수에 입력된 값이 자동으로 치환되어 수행됨
# invoke 변수가 여러개 일때는 dict형태로 넣어주어야한다.
chain.invoke({"country" : "대한민국"}).content

'대한민국의 수도는 서울입니다.'

### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성


추가 유효성 검사를 위해 `input_variables` 를 명시적으로 지정하세요.

이러한 변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하는 경우 예외를 발생시킵니다.


In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

- PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')
- 객체에서 필요한 값을넣어준다. from_template와 promptemplate의 차이


In [6]:
# prompt 생성
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [9]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [ ]:
prompt.format(country1="대한민국")

In [10]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country1', 'country2'], input_types={}, partial_variables={}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [11]:
# partial을 통해 partial 변수를 입력할 수 있다.
prompt_partial = prompt.partial(country2 = "캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [12]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [13]:
# chain으로 묶는다.
chain = prompt_partial | llm

In [14]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [15]:
# 덮어 쓴다
chain.invoke({"country1": "대한민국", "country2": "호주"}).content


'대한민국의 수도는 서울이고, 호주의 수도는 캔버라입니다.'

### `partial_variables`: 부분 변수 채움

`partial`을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것입니다. 이 사용 사례는 **항상 공통된 방식으로 가져오고 싶은 변수** 가 있는 경우입니다.

대표적인 예가 **날짜나 시간** 입니다.

항상 현재 날짜가 표시되기를 원하는 프롬프트가 있다고 가정해 보겠습니다. 프롬프트에 하드 코딩할 수도 없고, 다른 입력 변수와 함께 전달하는 것도 번거롭습니다. 이 경우 항상 현재 **날짜를 반환하는 함수** 를 사용하여 프롬프트를 부분적으로 변경할 수 있으면 매우 편리합니다.


다음의 코드는 오늘 날짜를 구하는 파이썬 코드입니다.


In [16]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'December 06'

In [17]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [18]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)

In [19]:
print(get_today())

December 06


In [20]:
# prompt 생성
prompt.format(n=3)

'오늘의 날짜는 December 06 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [21]:
# chain 을 생성합니다.
chain = prompt | llm

In [22]:
# chain 을 실행 후 결과를 확인합니다.
print(chain.invoke(3).content)

1. 데이브드 린치 (David Lynch) - 1946년 12월 06일
2. 조세 카레라스 (Jose Carreras) - 1946년 12월 06일
3. 리사 웰치 (Lisa Welch) - 1961년 12월 06일


In [23]:
# chain 을 실행 후 결과를 확인합니다.
# 기본값에서 dict형태로 입력하면 덮어쓴 상태에서 template을 진행하게 된다.
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

1. 김태희 (1980년 3월 29일)
2. 조승우 (1980년 3월 28일)
3. 배용준 (1983년 3월 28일)


## 파일로부터 template 읽어오기


```
_type: "prompt" # 다양한 타입이 들어가 있다.
template: "{fruit}의 색깔이 뭐야?"
input_variables: ["fruit"] # 변수를 설정한다
```

- 들여쓰기를 주의한다.

```
_type: "prompt"
template: |
  {country}의 수도에 대해서 알려주세요.
  수도의 특징을 다음의 양식에 맞게 정리해 주세요.
  300자 내외로 작성해 주세요.
  한글로 작성해 주세요.
  ----
  [양식]
  1. 면적
  2. 인구
  3. 역사적 장소
  4. 특산품

  #Answer:
input_variables: ["country"]
```


In [24]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

Window 사용자 중 이전의 코드가 오류가 나는 경우 아래의 코드로 실행하세요(인코딩 설정)


In [25]:
from langchain_teddynote.prompts import load_prompt

# Windows 사용자 only: 인코딩을 cp949로 설정
load_prompt("prompts/fruit_color.yaml", encoding="utf-8")

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [26]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [27]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



In [34]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt2 | llm 
# 아 topic을 invoke에 넣어야 하겠구나 라고 생각!

In [35]:
chain

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도에 대해서 알려주세요.\n수도의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 면적\n2. 인구\n3. 역사적 장소\n4. 특산품\n\n#Answer:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x13d1bb390>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x13e28cb50>, root_client=<openai.OpenAI object at 0x11d462e50>, root_async_client=<openai.AsyncOpenAI object at 0x13e25c4d0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [36]:
chain = prompt2 | llm | StrOutputParser()


In [40]:
answer = chain.invoke({"country" : "대한민국"})

In [41]:
answer

'1. 면적\n- 서울 특별시 면적은 약 605.21km²로 한반도 중앙에 위치하고 있다.\n\n2. 인구\n- 서울 특별시의 인구는 약 9,700만 명으로 대한민국의 인구의 약 1/5을 차지하고 있다.\n\n3. 역사적 장소\n- 경복궁, 창경궁, 덕수궁 등 다양한 조선 시대의 왕궁과 궁전이 위치해 있으며, 종묘, 선릉 등 역사적인 유적지도 많이 존재한다.\n\n4. 특산품\n- 한복, 한지, 불고기, 삼겹살, 떡볶이, 김치 등 다양한 대한민국 전통 음식과 문화 상품들이 수도인 서울에서 만날 수 있다.'

In [42]:
from langchain_teddynote.messages import stream_response

chain = prompt2 | ChatOpenAI(model_name = 'gpt-4o',temperature = 0) | StrOutputParser()

answer = chain.stream({"country" : "발리"})
stream_response(answer)


발리의 수도는 덴파사르입니다. 

1. 면적: 덴파사르의 면적은 약 123.98 제곱킬로미터로, 발리 섬의 남부에 위치해 있습니다.

2. 인구: 덴파사르는 발리에서 가장 인구가 많은 도시로, 약 90만 명 이상의 인구가 거주하고 있습니다.

3. 역사적 장소: 덴파사르에는 발리의 전통과 역사를 엿볼 수 있는 다양한 장소가 있습니다. 그 중 하나는 발리 박물관으로, 발리의 문화와 예술을 전시하고 있습니다. 또한, 푸푸탄 광장은 1906년 네덜란드 식민지 시대의 저항을 기념하는 장소로 유명합니다.

4. 특산품: 덴파사르는 발리 전통 공예품의 중심지로, 바틱 천과 은세공품이 유명합니다. 이 외에도 발리 전통 의상과 수공예품을 쉽게 찾아볼 수 있습니다.

- yaml 파일 형식으로 prompt를 정리해 놓으면 좋다!

## ChatPromptTemplate
- 대화형 형식으로 대화하기 좋은 template

`ChatPromptTemplate` 은 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있습니다.

메시지는 튜플(tuple) 형식으로 구성하며, (`role`, `message`) 로 구성하여 리스트로 생성할 수 있습니다.

**role**

- `"system"`: 시스템 설정 메시지 입니다. 주로 전역설정과 관련된 프롬프트입니다.
    - ai의 역할, 대화가 시작되면서, 끝날떄까지의 환경
- `"human"` : 사용자 입력 메시지 입니다.
    - 우리가 입력한 질문값
- `"ai"`: AI 의 답변 메시지입니다.


In [43]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

- messages 라는 값이 생성된다.

In [44]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

- Human 
- (role, message)
- messeage : AI답변, 역할, 전역prompt 이다.

In [45]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message 메세지는 튜플형식으로 넣는다.
        # 이전 템플렛은 단순한 질문
        # 대화형 형식의 template을 만들어 놓고 role이라는 역할이 있고
        # 답변이 있다.
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

생성한 메시지를 바로 주입하여 결과를 받을 수 있습니다.


In [46]:
llm.invoke(messages).content

'제 이름은 테디입니다. 어떻게 도와드릴까요?'

이번에는 체인을 생성해 보겠습니다.


In [47]:
chain = chat_template | llm

In [48]:
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content

'제 이름은 Teddy 입니다. 필요하신 도움이 있으신가요?'

## MessagePlaceholder

또한 LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 `MessagePlaceholder` 를 제공합니다.

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있습니다.


In [50]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"), # 아직확정된 메세지는 아니지만, 언젠간 넣어줄 데이터 // 쳇봇을 만들때 대화내용을 기록하고자 할때 messageholder로 // 대화자체도 지속적으로 잡아주는 // 아직확정되지 않은 대화를 넣어주기위해 
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."), # 여기는 꼭 채워주어여하는 변수가 있다.
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

`conversation` 대화목록을 나중에 추가하고자 할 때 `MessagesPlaceholder` 를 사용할 수 있습니다.


In [51]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [52]:
# chain 생성
chain = chat_prompt | llm | StrOutputParser()

In [53]:
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

'새로 입사한 테디 반가워요.'